In [2]:
import os
import pickle
import pandas as pd
import mygene

clusters=[]
files = os.listdir('/home/jain/clusters/sept5-clusters')
print(files)
os.chdir('/home/jain/clusters/sept5-clusters')
log_all = pd.read_csv('https://ftp.mcs.anl.gov/pub/candle/public/benchmarks/Pilot1/normal-tumor/matched_normal_samples.FPKM-UQ.log-transformed.csv', sep=',')
import mygene
mg = mygene.MyGeneInfo()

['cf_class_0_cluster1.pkl', 'cf_class_1_cluster1.pkl', 'cf_class_1_cluster0.pkl', 'cf_class_0_cluster0.pkl']


- Perturbation vectors are obtained by running Counterfactual analysis on NT3 dataset
- These are the files (pkl above). Each file represents 1 cluster of perturbation vectors. Using KMeans with 2D PCA.
- Inside each file is the centroid, positive indices (indices whos threshold is > 0.9max), negative indices (indices whos threshold is < -0.9max), all the sample indices (from the 1100 dataset) that belong to that particular cluster.
- The file names correspond to the cf class (0 or 1) and the cluster index.
- Here we try to do a literature review of the threshold genes for cancer
- The various types of cancer tumors used in our dataset are here: https://ftp.mcs.anl.gov/pub/candle/public/benchmarks/Pilot1/normal-tumor/matched_normal_samples.metadata.tsv

In [6]:
gene_all = list(log_all.columns.values)

In [33]:
# print(gene_all)

In [34]:
gene_all=gene_all[1:]

In [37]:
os.chdir('/home/jain/clusters/sept5-clusters')
cluster_genes=[]
for f in files:
    clusters = pickle.load(open(f, "rb"))
    print("Cluster: ", f)
#     print(clusters)
    print("+ve",clusters['positive threshold indices'])
    print("-ve",clusters['negative threshold indices'])
    a=clusters['positive threshold indices']
    b=clusters['negative threshold indices']
    for bb in b:
        for i in bb:
            print("print", i, type(i))
            cluster_genes.append(i)
    for aa in a:
        for j in aa:
            print("print", j, type(j))
            cluster_genes.append(j)
cluster_genes

Cluster:  cf_class_0_cluster1.pkl
+ve (array([], dtype=int64),)
-ve (array([58261]),)
print 58261 <class 'numpy.int64'>
Cluster:  cf_class_1_cluster1.pkl
+ve (array([], dtype=int64),)
-ve (array([ 8618,  9603, 14270, 32077]),)
print 8618 <class 'numpy.int64'>
print 9603 <class 'numpy.int64'>
print 14270 <class 'numpy.int64'>
print 32077 <class 'numpy.int64'>
Cluster:  cf_class_1_cluster0.pkl
+ve (array([], dtype=int64),)
-ve (array([ 9603, 32077]),)
print 9603 <class 'numpy.int64'>
print 32077 <class 'numpy.int64'>
Cluster:  cf_class_0_cluster0.pkl
+ve (array([], dtype=int64),)
-ve (array([12122]),)
print 12122 <class 'numpy.int64'>


[58261, 8618, 9603, 14270, 32077, 9603, 32077, 12122]

In [51]:
gene_names = []
# 9603 32077 appears twice? 
cluster_genes=list(set(cluster_genes))
for g in cluster_genes:
    split=gene_all[g].split(".")
    gene_names.append(split[0])
    
    
cluster_genes

[9603, 8618, 32077, 58261, 12122, 14270]

In [52]:
gene_names
geneSyms = mg.querymany(gene_names , scopes='ensembl.gene', fields='symbol', species='human')

querying 1-6...done.
Finished.


In [70]:
geneSyms

[{'query': 'ENSG00000236699',
  '_id': '54848',
  '_score': 22.490173,
  'symbol': 'ARHGEF38'},
 {'query': 'ENSG00000223516',
  '_id': 'ENSG00000223516',
  '_score': 23.34138,
  'symbol': 'AFF2-IT1'},
 {'query': 'ENSG00000173826',
  '_id': '81033',
  '_score': 22.549414,
  'symbol': 'KCNH6'},
 {'query': 'ENSG00000143344',
  '_id': '23179',
  '_score': 22.027897,
  'symbol': 'RGL1'},
 {'query': 'ENSG00000159387',
  '_id': '79190',
  '_score': 22.549414,
  'symbol': 'IRX6'},
 {'query': 'ENSG00000207060',
  '_id': 'ENSG00000207060',
  '_score': 7.529477,
  'symbol': 'RNU6-480P'}]

In [89]:
symbols = [i['symbol'] for i in geneSyms if 'symbol' in i]
symbols

['ARHGEF38', 'AFF2-IT1', 'KCNH6', 'RGL1', 'IRX6', 'RNU6-480P']

In [64]:
geneSyms[:][0] 
# appeared twice in our list

{'query': 'ENSG00000236699',
 '_id': '54848',
 '_score': 22.490173,
 'symbol': 'ARHGEF38'}

The results from the present study demonstrated that ARHGEF38 is significantly overexpressed in PCa than BPH, especially, high-grade prostate cancer. This suggested that ARHGEF38 might promote prostate cancer migration and hence attribute to PCa progression. https://www.proteinatlas.org/ENSG00000236699-ARHGEF38/pathology

In [65]:
geneSyms[:][1]

{'query': 'ENSG00000223516',
 '_id': 'ENSG00000223516',
 '_score': 23.34138,
 'symbol': 'AFF2-IT1'}

Related to mental health https://www.genecards.org/cgi-bin/carddisp.pl?gene=AFF2
Maybe thyroid cancer https://www.proteinatlas.org/ENSG00000155966-AFF2/pathology

In [66]:
geneSyms[:][2]
# appeared twice in our list

{'query': 'ENSG00000173826',
 '_id': '81033',
 '_score': 22.549414,
 'symbol': 'KCNH6'}

https://www.proteinatlas.org/ENSG00000173826-KCNH6/tissue 
https://www.proteinatlas.org/ENSG00000173826-KCNH6/pathology (prostate cancer) graph shows elevated values

In [67]:
geneSyms[:][3]

{'query': 'ENSG00000143344',
 '_id': '23179',
 '_score': 22.027897,
 'symbol': 'RGL1'}

https://www.proteinatlas.org/ENSG00000143344-RGL1/pathology no specific type of cancer but graph shows can be present at elevated levels for several types of cancer

In [68]:
geneSyms[:][4]

{'query': 'ENSG00000159387',
 '_id': '79190',
 '_score': 22.549414,
 'symbol': 'IRX6'}

https://www.proteinatlas.org/ENSG00000159387-IRX6/pathology posible lung cancer

In [69]:
geneSyms[:][5]

{'query': 'ENSG00000207060',
 '_id': 'ENSG00000207060',
 '_score': 7.529477,
 'symbol': 'RNU6-480P'}

might be related to colon cancer https://rgd.mcw.edu/rgdweb/report/gene/main.html?id=1343798 